In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# import the required libraries
import numpy as np
import pickle
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.regularizers import *
from tensorflow.keras import backend as keras
import tensorflow as tf
from PIL import Image, ImageOps
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import pandas as pd

Mounted at /content/drive


In [ ]:
from tqdm import tqdm

In [ ]:
import matplotlib.patheffects as pe

In [ ]:
x_test = np.load('/content/drive/MyDrive/Disk/Without_Noise/xtest_16_17.npz')
x_test = x_test['arr_0']
x_test = x_test.astype('float32')

y_test = np.load('/content/drive/MyDrive/Disk/Without_Noise/ytest_16_17.npz')
y_test = y_test['arr_0']
y_test = y_test.astype('float32')

In [ ]:
num_img = y_test.shape[0]
m_test = y_test.shape[1]
n_test = y_test.shape[2]
num_img

119

In [ ]:
max_val_den = 1e19
max_val_erms = 13.32106

In [ ]:
y_test /= 255.0
y_test *= max_val_den

In [ ]:
def UNet_MCD_1(input_size, reg_fn):
    dr = np.random.uniform(0,0.1)

    inputs = Input(shape=input_size)

    conv1 = Conv2D(8, 3, activation='relu', padding='same', kernel_regularizer=reg_fn)(inputs)
    # conv1 = Dropout(dropout_rate)(conv1, training=True)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(16, 3, activation='relu', padding='same', kernel_regularizer=reg_fn)(pool1)
    # conv2 = Dropout(dropout_rate)(conv2, training=True)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(32, 3, activation='relu', padding='same', kernel_regularizer=reg_fn)(pool2)
    # conv3 = Dropout(dropout_rate)(conv3, training=True)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(64, 3, activation='relu', padding='same', kernel_regularizer=reg_fn)(pool3)
    conv4 = Dropout(dr)(conv4, training=True)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(128, 3, activation='relu', padding='same', kernel_regularizer=reg_fn)(pool4)
    conv5 = Dropout(dr)(conv5, training=True)
    pool5 = MaxPooling2D(pool_size=(2, 2))(conv5)

    conv6 = Conv2D(256, 3, activation='relu', padding='same', kernel_regularizer=reg_fn)(pool5)
    conv6 = Dropout(dr)(conv6, training=True)
    pool6 = MaxPooling2D(pool_size=(2, 2))(conv6)
      
    up7 = Conv2DTranspose(128, 2, strides=(2, 2), activation='relu', padding='same', kernel_regularizer=reg_fn)(pool6)
    up7 = Dropout(dr)(up7, training=True)
    sc7 = Concatenate()([up7, pool5])

    up8 = Conv2DTranspose(64, 2, strides=(2, 2), activation='relu', padding='same', kernel_regularizer=reg_fn)(sc7)
    up8 = Dropout(dr)(up8, training=True)
    sc8 = Concatenate()([up8, pool4])

    up9 = Conv2DTranspose(32, 2, strides=(2, 2), activation='relu', padding='same', kernel_regularizer=reg_fn)(sc8)
    # up9 = Dropout()(up9, training=True)
    sc9 = Concatenate()([up9, pool3])

    up10 = Conv2DTranspose(16, 2, strides=(2, 2), activation='relu', padding='same', kernel_regularizer=reg_fn)(sc9)
    # up10 = Dropout()(up10, training=True)
    sc10 = Concatenate()([up10, pool2])

    up11 = Conv2DTranspose(8, 2, strides=(2, 2), activation='relu', padding='same', kernel_regularizer=reg_fn)(sc10)
    # up11 = Dropout()(up11, training=True)
    sc11 = Concatenate()([up11, pool1])

    outputs = Conv2DTranspose(1, 2, strides=(2, 2), activation='relu', padding='same', kernel_regularizer=reg_fn)(sc11)
    
    model = Model(inputs, outputs)

    model.compile(optimizer='adam', metrics=['accuracy'], loss=tf.keras.losses.MeanSquaredError())

    return model

In [ ]:
model = UNet_MCD_1((256,256,1),l1(1e-6))
model.load_weights('/content/drive/MyDrive/Disk/Without_Noise/MC_P_1/1e-06_MCD_P_1.h5')

In [ ]:
test = np.empty((1,256,256,1))
test[0] = x_test[14]

In [ ]:
op = np.empty((300,256,256,1))
for i in range(300):
  op[i] = model.predict(test, verbose=0)


In [ ]:
op /= 255.0
op *= max_val_den

mean = np.mean(op,axis=0)
std = np.std(op,axis=0)

Mean = mean[int(m_test/2)]
Std = std[int(m_test/2)]

In [ ]:
Up_1 = Mean + Std
Low_1 = Mean - Std

Up = Mean + 2*Std
Low = Mean - 2*Std

plt.figure()
plt.title("Density Profile", fontsize=15)
plt.xlabel("No. of cells along $x_{central}$", fontsize=15)
plt.ylabel("Density", fontsize=15)  
plt.fill_between(np.arange(m_test), Low.squeeze(),Up.squeeze(), color="#ff72be",label="$\mu \pm 2\sigma$")
plt.fill_between(np.arange(m_test), Low_1.squeeze(),Up_1.squeeze(), color="#9b5999",label="$\mu \pm \sigma$")
plt.plot(y_test[14][int(m_test/2)], color='cyan', lw=1, path_effects=[pe.Stroke(linewidth=2.5, foreground='navy'), pe.Normal()], label="Ground Truth")
plt.legend()

In [ ]:
!mkdir Density\ Profile\ 1e16-1e17

In [ ]:
for ii in range(num_img):
  test = []
  test.append(x_test[ii])
  test = np.array(test)

  op = np.empty((300,256,256,1))
  for i in tqdm(range(300)):
    op[i] = model.predict(test, verbose = 0)

  op /= 255.0
  op *= max_val_den

  mean = np.mean(op,axis=0)
  std = np.std(op,axis=0)

  Mean = mean[int(m_test/2)]
  Std = std[int(m_test/2)]

  Up_1 = Mean + Std
  Low_1 = Mean - Std

  Up = Mean + 2*Std
  Low = Mean - 2*Std

  plt.figure()
  plt.title("Density Profile", fontsize=15)
  plt.xlabel("No. of cells along $x_{central}$", fontsize=15)
  plt.ylabel("Density", fontsize=15)  
  plt.fill_between(np.arange(m_test), Low.squeeze(),Up.squeeze(), color="#ff72be",label="$\mu \pm 2\sigma$")
  plt.fill_between(np.arange(m_test), Low_1.squeeze(),Up_1.squeeze(), color="#9b5999",label="$\mu \pm \sigma$")
  plt.plot(y_test[ii][int(m_test/2)], color='cyan', lw=1, path_effects=[pe.Stroke(linewidth=2.5, foreground='navy'), pe.Normal()], label="Ground Truth")
  plt.legend()
  plt.savefig("/content/Density Profile 1e16-1e17/Sample "+str(ii+1)+".png")
  plt.close()

100%|██████████| 300/300 [00:31<00:00,  9.55it/s]


In [ ]:

!zip -r /content/Density\ Profile\ 1e16-1e17.zip /content/Density\ Profile\ 1e16-1e17

  adding: content/Density Profile 1e16-1e17/ (stored 0%)
  adding: content/Density Profile 1e16-1e17/Sample 109.png (deflated 8%)
  adding: content/Density Profile 1e16-1e17/Sample 55.png (deflated 8%)
  adding: content/Density Profile 1e16-1e17/Sample 4.png (deflated 7%)
  adding: content/Density Profile 1e16-1e17/Sample 32.png (deflated 7%)
  adding: content/Density Profile 1e16-1e17/Sample 61.png (deflated 8%)
  adding: content/Density Profile 1e16-1e17/Sample 53.png (deflated 8%)
  adding: content/Density Profile 1e16-1e17/Sample 41.png (deflated 8%)
  adding: content/Density Profile 1e16-1e17/Sample 11.png (deflated 7%)
  adding: content/Density Profile 1e16-1e17/Sample 44.png (deflated 7%)
  adding: content/Density Profile 1e16-1e17/Sample 36.png (deflated 7%)
  adding: content/Density Profile 1e16-1e17/Sample 35.png (deflated 8%)
  adding: content/Density Profile 1e16-1e17/Sample 108.png (deflated 7%)
  adding: content/Density Profile 1e16-1e17/Sample 92.png (deflated 8%)
  addi